### Asignment 3
* Add regularization to your CFAR-10 model (Dropout, L1, and L2)
* Find out how deep the model can be and still improving
* Draw confusion matrix
* Use five different optimizers and compare their results and convergence time (SGD, Adam, Adagrad, Adadelta, RMSprop)
* Use three different activation functions (ReLu, Tanh, Sigmoid)
* Draw all of your experiments in tensorboard or wandb!
* Write softmax loss and Hinge loss without using torch.nn


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.datasets as dsets
import random as rand
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import time
from torch.autograd import Variable
from tabulate import tabulate

In [ ]:
if torch.cuda.is_available():
    avDev = torch.device("cuda")
else:
    avDev = torch.device("cpu")

print(avDev)

cuda


Loading the dataset from CIFAR-10 dataset

In [ ]:
trainset = dsets.CIFAR10('./data', train=True, download=True, transform=transforms.ToTensor())
testset = dsets.CIFAR10('./data', train=False, download=True, transform=transforms.ToTensor())

Files already downloaded and verified
Files already downloaded and verified


Intializing the Hyper Parameters

In [ ]:
torch.manual_seed(123)
batch_size = 100
n_iters = 10000
num_epochs = n_iters / (len(trainset) / batch_size)
num_epochs = int(num_epochs)
print(num_epochs)

hidden_layer = 1000
learning_rate = 0.01
momentum = 0.5


20


Lists to store model's accuracy and convergence times for diiferent optimizers


In [ ]:
list1 = [['Optmizer with Regularisation','Accuracy','Convergence time(sec)']]

Implementing the Logistic Regression class inherited from NN.Module super class

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(3*32*32, 1000)
        self.fc1_drop = nn.Dropout(0.2)
        self.fc2 = nn.Linear(1000, 100)
        self.fc2_drop = nn.Dropout(0.2)
        self.fc3 = nn.Linear(100, 10)

    def forward(self, x):
        x = x.view(-1, 3*32*32)
        x = torch.relu(self.fc1(x))
        x = self.fc1_drop(x)
        x = torch.relu(self.fc2(x))
        x = self.fc2_drop(x)
        return F.log_softmax(self.fc3(x),dim=1)

Loading training and testing data to the loader + intializing the model + putting the model & the loss function into the GPU

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=trainset, 
                                           batch_size=batch_size, 
                                           shuffle=True)
 
test_loader = torch.utils.data.DataLoader(dataset=testset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

shape = trainset[0][0].numpy().shape
input_dim = shape[0] * shape[1] * shape[2]
output_dim = 10
 
model = Net()

model.to(avDev)

Net(
  (fc1): Linear(in_features=3072, out_features=1000, bias=True)
  (fc1_drop): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=1000, out_features=100, bias=True)
  (fc2_drop): Dropout(p=0.2, inplace=False)
  (fc3): Linear(in_features=100, out_features=10, bias=True)
)

Training and Testing the model

In [ ]:
def train(model,reg, optimizer, epoch, log_interval=100):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if avDev:
            data, target = data.cuda(), target.cuda()
        optimizer.zero_grad()
        output = model(data)
        
        if reg:
          lamda=0.8
          l1_reg=0
        
          for W in model_l1.parameters():
            W=Variable(W,requires_grad=True)
            l1_reg += torch.sum(torch.abs(W))
          loss = F.nll_loss(output, target) + lamda*l1_reg
        else:
          loss = F.nll_loss(output, target)
        
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * float(batch_idx) / len(train_loader), loss.item()))
            
def test(model,loss_vector, accuracy_vector):
    model.eval()
    test_loss, correct = 0, 0
    confusion_matrix = np.zeros((10,10))
    for data, target in test_loader:
        if avDev:
            data, target = data.cuda(), target.cuda()
        data.requires_grad_(False)#We use to use Volatile!
        target.requires_grad_(False)
        output = model(data)
        test_loss += F.nll_loss(output, target).item()
        pred = output.data.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum().float()
        for i in range(50):
          confusion_matrix[pred[i]][target[i]]+=1
    
    test_loss /= len(test_loader)
    loss_vector.append(test_loss)
    
    accuracy = 100. * correct / float(len(test_loader.dataset))
    accuracy_vector.append(accuracy)
    
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset), float(accuracy)))
    print ( "confusion matrix = ",confusion_matrix )


SGD, Dropout

In [ ]:
epochs = 10
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)
arr = ['SGD-Dropout']
start = time.time()
lossv, accv = [], []
for epoch in range(1, epochs + 1):
    train(model,False,optimizer, epoch)
    test(model,lossv, accv)
end = time.time()

arr.append(round(float(accv[-1]),2))
arr.append(round(float(end-start),2))
list1.append(arr)


Train Epoch: 1 [0/50000 (0%)]	Loss: 2.310077
Train Epoch: 1 [10000/50000 (20%)]	Loss: 2.255386
Train Epoch: 1 [20000/50000 (40%)]	Loss: 2.172878
Train Epoch: 1 [30000/50000 (60%)]	Loss: 2.009183
Train Epoch: 1 [40000/50000 (80%)]	Loss: 1.914672

Test set: Average loss: 1.9385, Accuracy: 3083.0/10000 (30.83%)

confusion matrix =  [[227.  36.  92.  44.  65.  52.  23.  51.  72.  33.]
 [ 15. 127.  30.  37.  34.  28.  42.  32.  24.  60.]
 [  0.   1.  28.   9.  18.   8.   4.   3.   1.   0.]
 [  0.   0.   0.   1.   0.   0.   0.   0.   0.   0.]
 [  0.   1.  23.   8.  35.  11.   6.  19.   0.   0.]
 [ 18.  33.  95. 170. 110. 215.  97.  92.  33.  11.]
 [ 14.  44. 119.  82. 156.  83. 232.  56.   3.  25.]
 [ 19.  12.  52.  61.  55.  50.  45. 116.   4.  13.]
 [152. 133.  47.  50.  36.  46.  12.  44. 310. 104.]
 [ 28. 124.  18.  48.  19.  22.  39.  72.  61. 220.]]
Train Epoch: 2 [0/50000 (0%)]	Loss: 2.062369
Train Epoch: 2 [10000/50000 (20%)]	Loss: 1.958951
Train Epoch: 2 [20000/50000 (40%)]	Loss: 1.

ADAM with Dropout

In [ ]:
epochs = 10
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
start = time.time()
lossv, accv = [], []
for epoch in range(1, epochs + 1):
    train(model,False,optimizer, epoch)
    test(model,lossv, accv)
end = time.time()
arr = ['Adam - Dropout']
arr.append(round(float(accv[-1]),2))
arr.append(round(float(end-start),2))
list1.append(arr)

Train Epoch: 1 [0/50000 (0%)]	Loss: 1.448259
Train Epoch: 1 [10000/50000 (20%)]	Loss: 2.320598
Train Epoch: 1 [20000/50000 (40%)]	Loss: 2.303191
Train Epoch: 1 [30000/50000 (60%)]	Loss: 2.310070
Train Epoch: 1 [40000/50000 (80%)]	Loss: 2.304879

Test set: Average loss: 2.3033, Accuracy: 1000.0/10000 (10.00%)

confusion matrix =  [[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [473. 511. 504. 510. 528. 515. 500. 485. 508. 466.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]]
Train Epoch: 2 [0/50000 (0%)]	Loss: 2.306486
Train Epoch: 2 [10000/50000 (20%)]	Loss: 2.306542
Train Epoch: 2 [20000/50000 (40%)]	Loss: 2.

Adagrad

In [ ]:
optimizer = torch.optim.Adagrad(model.parameters(), lr=0.01, lr_decay=0.001, weight_decay=0.01, initial_accumulator_value=0)
start = time.time()
lossv, accv = [], []
for epoch in range(1, epochs + 1):
    train(model,False,optimizer, epoch)
    test(model,lossv, accv)
end = time.time()
arr = ['Adagrad - Dropout']
arr.append(round(float(accv[-1]),2))
arr.append(round(float(end-start),2))
list1.append(arr)

Train Epoch: 1 [0/50000 (0%)]	Loss: 2.306135
Train Epoch: 1 [10000/50000 (20%)]	Loss: 2.303179
Train Epoch: 1 [20000/50000 (40%)]	Loss: 2.292916
Train Epoch: 1 [30000/50000 (60%)]	Loss: 2.179269
Train Epoch: 1 [40000/50000 (80%)]	Loss: 2.146790

Test set: Average loss: 2.0746, Accuracy: 1827.0/10000 (18.27%)

confusion matrix =  [[ 84.  55.  48.  57.  37.  65.  35.  75.  65.  57.]
 [  1.   5.   3.   0.   1.   2.   1.   0.   1.   1.]
 [ 58.  59.  77. 100.  86. 100.  63.  85.  37.  42.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [ 24.  27.  76. 109.  70.  95.  92.  81.   9.  25.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [ 20.  50. 208. 178. 270. 214. 277. 161.  17.  19.]
 [  0.   1.   0.   0.   0.   0.   0.   0.   0.   0.]
 [286. 314.  92.  66.  64.  39.  32.  83. 379. 322.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]]
Train Epoch: 2 [0/50000 (0%)]	Loss: 2.059215
Train Epoch: 2 [10000/50000 (20%)]	Loss: 2.070028
Train Epoch: 2 [20000/50000 (40%)]	Loss: 2.

Adadelta

In [ ]:
epochs = 10
optimizer = torch.optim.Adadelta(model.parameters(), lr=1.0, rho=0.9, eps=1e-06, weight_decay=0.1)
start = time.time()
lossv, accv = [], []
for epoch in range(1, epochs + 1):
    train(model,False,optimizer, epoch)
    test(model,lossv, accv)
end = time.time()
arr = ['Adadelta - Dropout']
arr.append(round(float(accv[-1]),2))
arr.append(round(float(end-start),2))
list1.append(arr)

Train Epoch: 1 [0/50000 (0%)]	Loss: 2.165120
Train Epoch: 1 [10000/50000 (20%)]	Loss: 2.210075
Train Epoch: 1 [20000/50000 (40%)]	Loss: 2.232720
Train Epoch: 1 [30000/50000 (60%)]	Loss: 2.250237
Train Epoch: 1 [40000/50000 (80%)]	Loss: 2.204685

Test set: Average loss: 2.2300, Accuracy: 1319.0/10000 (13.19%)

confusion matrix =  [[  0.   0.   1.   0.   0.   0.   0.   0.   0.   0.]
 [ 25.  51. 116. 108. 112. 110. 138.  90.  23.  27.]
 [  0.   0.   1.   0.   1.   1.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   4.   3.   7.   3.   2.   1.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [ 14.  38.  93. 126. 174. 132. 154. 111.  20.  17.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [434. 422. 293. 272. 238. 265. 205. 282. 464. 422.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]]
Train Epoch: 2 [0/50000 (0%)]	Loss: 2.274870
Train Epoch: 2 [10000/50000 (20%)]	Loss: 2.248648
Train Epoch: 2 [20000/50000 (40%)]	Loss: 2.

RMSProp with dropout

In [ ]:
optimizer = torch.optim.RMSprop(model.parameters(), lr=0.01, alpha=0.99, eps=1e-08, weight_decay=0.1, momentum=0.9, centered=True)
start = time.time()
lossv, accv = [], []
for epoch in range(1, epochs + 1):
    train(model,False,optimizer, epoch)
    test(model,lossv, accv)
end = time.time()
arr = ['RMSProp - Dropout']
arr.append(round(float(accv[-1]),2))
arr.append(round(float(end-start),2))
list1.append(arr)

Train Epoch: 1 [0/50000 (0%)]	Loss: 2.279445
Train Epoch: 1 [10000/50000 (20%)]	Loss: 4.797539
Train Epoch: 1 [20000/50000 (40%)]	Loss: 2.621446
Train Epoch: 1 [30000/50000 (60%)]	Loss: 2.299710
Train Epoch: 1 [40000/50000 (80%)]	Loss: 2.290968

Test set: Average loss: 2.2956, Accuracy: 1238.0/10000 (12.38%)

confusion matrix =  [[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [241. 155.  96.  61.  50.  61.  44. 106. 261. 215.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [193. 318. 407. 445. 478. 454. 455. 372. 185. 200.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [ 39.  38.   1.   4.   0.   0.   1.   7.  62.  51.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]]
Train Epoch: 2 [0/50000 (0%)]	Loss: 2.300245
Train Epoch: 2 [10000/50000 (20%)]	Loss: 2.297894
Train Epoch: 2 [20000/50000 (40%)]	Loss: 2.

L1 regularisation


In [ ]:
class Net_l1(nn.Module):
    def __init__(self):
        super(Net_l1,self).__init__()
        self.fc1 = nn.Linear(3*32*32, 1000)
        self.fc2 = nn.Linear(1000, 100)
        self.fc3 = nn.Linear(100, 10)

    def forward(self,x):
        x = x.view(-1, 3*32*32)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        out = F.log_softmax(self.fc3(x),dim=1)
        return out

model_l1 = Net_l1()
model_l1.to(avDev)
epochs = 10


SGD, L1 regularisation

In [ ]:
optimizer = torch.optim.SGD(model_l1.parameters(), lr=learning_rate, momentum=momentum)
start = time.time()
lossv, accv = [], []
for epoch in range(1, epochs + 1):
    train(model_l1,True,optimizer, epoch)
    test(model_l1,lossv, accv)
end = time.time()
arr = ['SGD - L1']
arr.append(round(float(accv[-1]),2))
arr.append(round(float(end-start),2))
list1.append(arr)


Train Epoch: 1 [0/50000 (0%)]	Loss: 23490.179688
Train Epoch: 1 [10000/50000 (20%)]	Loss: 23497.416016
Train Epoch: 1 [20000/50000 (40%)]	Loss: 23516.646484
Train Epoch: 1 [30000/50000 (60%)]	Loss: 23535.904297
Train Epoch: 1 [40000/50000 (80%)]	Loss: 23554.044922

Test set: Average loss: 1.9258, Accuracy: 3169.0/10000 (31.69%)

confusion matrix =  [[210.  30.  76.  28.  45.  20.   6.  25. 124.  31.]
 [ 30. 169.  28.  36.  26.  24.  27.  31.  49.  78.]
 [ 25.   2.  28.  12.  22.  22.  10.   9.   4.   0.]
 [  1.   1.   3.   4.   2.   0.   0.   0.   0.   0.]
 [ 20.  15.  99.  67. 127.  71.  32. 102.  10.   8.]
 [ 15.  37.  41. 128.  41. 181.  51.  37.  43.  13.]
 [ 24.  62. 180. 136. 212. 136. 319. 102.   9.  48.]
 [ 35.  17.  21.  55.  30.  37.  27. 110.   9.  18.]
 [ 63.  43.  11.   5.  10.  11.   3.  10. 154.  27.]
 [ 50. 135.  17.  39.  13.  13.  25.  59. 106. 243.]]
Train Epoch: 2 [0/50000 (0%)]	Loss: 23570.308594
Train Epoch: 2 [10000/50000 (20%)]	Loss: 23586.023438
Train Epoch: 2 

Adam, L1 regulariation

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
start = time.time()
lossv, accv = [], []
for epoch in range(1, epochs + 1):
    train(model_l1,True,optimizer, epoch)
    test(model_l1,lossv, accv)
end = time.time()
arr = ['Adam - L1']
arr.append(round(float(accv[-1]),2))
arr.append(round(float(end-start),2))
list1.append(arr)

Train Epoch: 1 [0/50000 (0%)]	Loss: 24274.687500
Train Epoch: 1 [10000/50000 (20%)]	Loss: 24274.705078
Train Epoch: 1 [20000/50000 (40%)]	Loss: 24274.691406
Train Epoch: 1 [30000/50000 (60%)]	Loss: 24274.548828
Train Epoch: 1 [40000/50000 (80%)]	Loss: 24274.718750

Test set: Average loss: 1.5334, Accuracy: 4588.0/10000 (45.88%)

confusion matrix =  [[268.  26.  30.  10.  31.   5.   1.  16.  97.  22.]
 [  5. 212.   6.   1.   1.   0.   2.   2.  21.  42.]
 [ 49.   8. 185.  44. 103.  69.  55.  28.  23.   5.]
 [ 24.  29.  44. 194.  29. 100.  76.  36.  23.  41.]
 [ 16.  10.  65.  19. 189.  19.  58.  36.  17.   7.]
 [ 20.  27.  73. 145.  61. 236.  53.  57.  27.  18.]
 [  7.  11.  58.  31.  49.  30. 228.  12.   3.  15.]
 [ 26.  34.  35.  40.  51.  42.  19. 276.  14.  33.]
 [ 32.  24.   5.   7.   7.   8.   2.   4. 225.  10.]
 [ 26. 130.   3.  19.   7.   6.   6.  18.  58. 273.]]
Train Epoch: 2 [0/50000 (0%)]	Loss: 24274.726562
Train Epoch: 2 [10000/50000 (20%)]	Loss: 24274.656250
Train Epoch: 2 

Adagrad, L1 regularisation

In [ ]:
optimizer = torch.optim.Adagrad(model.parameters(), lr=0.01, lr_decay=0.001, weight_decay=0.01, initial_accumulator_value=0)
start = time.time()
lossv, accv = [], []
for epoch in range(1, epochs + 1):
    train(model_l1,True,optimizer, epoch)
    test(model_l1,lossv, accv)
end = time.time()
arr = ['Adagrad - L1']
arr.append(round(float(accv[-1]),2))
arr.append(round(float(end-start),2))
list1.append(arr)

Train Epoch: 1 [0/50000 (0%)]	Loss: 24274.785156
Train Epoch: 1 [10000/50000 (20%)]	Loss: 24274.767578
Train Epoch: 1 [20000/50000 (40%)]	Loss: 24274.738281
Train Epoch: 1 [30000/50000 (60%)]	Loss: 24274.677734
Train Epoch: 1 [40000/50000 (80%)]	Loss: 24274.708984

Test set: Average loss: 1.5334, Accuracy: 4588.0/10000 (45.88%)

confusion matrix =  [[268.  26.  30.  10.  31.   5.   1.  16.  97.  22.]
 [  5. 212.   6.   1.   1.   0.   2.   2.  21.  42.]
 [ 49.   8. 185.  44. 103.  69.  55.  28.  23.   5.]
 [ 24.  29.  44. 194.  29. 100.  76.  36.  23.  41.]
 [ 16.  10.  65.  19. 189.  19.  58.  36.  17.   7.]
 [ 20.  27.  73. 145.  61. 236.  53.  57.  27.  18.]
 [  7.  11.  58.  31.  49.  30. 228.  12.   3.  15.]
 [ 26.  34.  35.  40.  51.  42.  19. 276.  14.  33.]
 [ 32.  24.   5.   7.   7.   8.   2.   4. 225.  10.]
 [ 26. 130.   3.  19.   7.   6.   6.  18.  58. 273.]]
Train Epoch: 2 [0/50000 (0%)]	Loss: 24274.773438
Train Epoch: 2 [10000/50000 (20%)]	Loss: 24274.839844
Train Epoch: 2 

Adadelta, L1 regularistion


In [ ]:
optimizer = torch.optim.Adadelta(model.parameters(), lr=1.0, rho=0.9, eps=1e-06, weight_decay=0.1)
start = time.time()
lossv, accv = [], []
for epoch in range(1, epochs + 1):
    train(model_l1,True,optimizer, epoch)
    test(model_l1,lossv, accv)
end = time.time()
arr = ['Adadelta - L1']
arr.append(round(float(accv[-1]),2))
arr.append(round(float(end-start),2))
list1.append(arr)

Train Epoch: 1 [0/50000 (0%)]	Loss: 24274.656250
Train Epoch: 1 [10000/50000 (20%)]	Loss: 24274.865234
Train Epoch: 1 [20000/50000 (40%)]	Loss: 24274.736328
Train Epoch: 1 [30000/50000 (60%)]	Loss: 24274.828125
Train Epoch: 1 [40000/50000 (80%)]	Loss: 24274.869141

Test set: Average loss: 1.5334, Accuracy: 4588.0/10000 (45.88%)

confusion matrix =  [[268.  26.  30.  10.  31.   5.   1.  16.  97.  22.]
 [  5. 212.   6.   1.   1.   0.   2.   2.  21.  42.]
 [ 49.   8. 185.  44. 103.  69.  55.  28.  23.   5.]
 [ 24.  29.  44. 194.  29. 100.  76.  36.  23.  41.]
 [ 16.  10.  65.  19. 189.  19.  58.  36.  17.   7.]
 [ 20.  27.  73. 145.  61. 236.  53.  57.  27.  18.]
 [  7.  11.  58.  31.  49.  30. 228.  12.   3.  15.]
 [ 26.  34.  35.  40.  51.  42.  19. 276.  14.  33.]
 [ 32.  24.   5.   7.   7.   8.   2.   4. 225.  10.]
 [ 26. 130.   3.  19.   7.   6.   6.  18.  58. 273.]]
Train Epoch: 2 [0/50000 (0%)]	Loss: 24274.812500
Train Epoch: 2 [10000/50000 (20%)]	Loss: 24274.683594
Train Epoch: 2 

RMSProp, L1 regularisation

In [ ]:
optimizer = torch.optim.RMSprop(model.parameters(), lr=0.01, alpha=0.99, eps=1e-08, weight_decay=0.1, momentum=0.9, centered=True)
start = time.time()
lossv, accv = [], []
for epoch in range(1, epochs + 1):
    train(model_l1,True,optimizer, epoch)
    test(model_l1,lossv, accv)
end = time.time()
arr = ['RMSProp - L1']
arr.append(round(float(accv[-1]),2))
arr.append(round(float(end-start),2))
list1.append(arr)


Train Epoch: 1 [0/50000 (0%)]	Loss: 24274.863281
Train Epoch: 1 [10000/50000 (20%)]	Loss: 24274.867188
Train Epoch: 1 [20000/50000 (40%)]	Loss: 24275.027344
Train Epoch: 1 [30000/50000 (60%)]	Loss: 24274.636719
Train Epoch: 1 [40000/50000 (80%)]	Loss: 24274.736328

Test set: Average loss: 1.5334, Accuracy: 4588.0/10000 (45.88%)

confusion matrix =  [[268.  26.  30.  10.  31.   5.   1.  16.  97.  22.]
 [  5. 212.   6.   1.   1.   0.   2.   2.  21.  42.]
 [ 49.   8. 185.  44. 103.  69.  55.  28.  23.   5.]
 [ 24.  29.  44. 194.  29. 100.  76.  36.  23.  41.]
 [ 16.  10.  65.  19. 189.  19.  58.  36.  17.   7.]
 [ 20.  27.  73. 145.  61. 236.  53.  57.  27.  18.]
 [  7.  11.  58.  31.  49.  30. 228.  12.   3.  15.]
 [ 26.  34.  35.  40.  51.  42.  19. 276.  14.  33.]
 [ 32.  24.   5.   7.   7.   8.   2.   4. 225.  10.]
 [ 26. 130.   3.  19.   7.   6.   6.  18.  58. 273.]]
Train Epoch: 2 [0/50000 (0%)]	Loss: 24274.572266
Train Epoch: 2 [10000/50000 (20%)]	Loss: 24274.912109
Train Epoch: 2 

L2 Regularisation - handled by weight_decay parameter

In [ ]:
model = Net_l1()
model.to(avDev)

Net_l1(
  (fc1): Linear(in_features=3072, out_features=1000, bias=True)
  (fc2): Linear(in_features=1000, out_features=100, bias=True)
  (fc3): Linear(in_features=100, out_features=10, bias=True)
)

SGD, L2 regularisation

In [ ]:
optimizer = torch.optim.SGD(model_l1.parameters(), lr=learning_rate, momentum=momentum, weight_decay=0.1)
start = time.time()
lossv, accv = [], []
for epoch in range(1, epochs + 1):
    train(model,False,optimizer, epoch)
    test(model,lossv, accv)
end = time.time()
arr = ['SGD - L2']
arr.append(round(float(accv[-1]),2))
arr.append(round(float(end-start),2))
list1.append(arr)


Train Epoch: 1 [0/50000 (0%)]	Loss: 2.308578
Train Epoch: 1 [10000/50000 (20%)]	Loss: 2.296941
Train Epoch: 1 [20000/50000 (40%)]	Loss: 2.300027
Train Epoch: 1 [30000/50000 (60%)]	Loss: 2.313859
Train Epoch: 1 [40000/50000 (80%)]	Loss: 2.302550

Test set: Average loss: 2.3041, Accuracy: 972.0/10000 (9.72%)

confusion matrix =  [[  0.   0.   0.   0.   0.   0.   0.   0.   1.   0.]
 [124. 124. 144. 187. 165. 167. 181. 161. 139. 135.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [214. 264. 256. 200. 278. 223. 248. 227. 289. 246.]
 [  3.   8.   5.  12.   1.  22.   2.   4.   2.   4.]
 [128. 109.  90. 101.  76.  97.  64.  82.  73.  76.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  3.   4.   2.   6.   6.   2.   1.   5.   4.   4.]
 [  1.   2.   7.   4.   2.   4.   4.   6.   0.   1.]]
Train Epoch: 2 [0/50000 (0%)]	Loss: 2.301853
Train Epoch: 2 [10000/50000 (20%)]	Loss: 2.295821
Train Epoch: 2 [20000/50000 (40%)]	Loss: 2.30

Adam, L2 regulariation

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.1)
start = time.time()
lossv, accv = [], []
for epoch in range(1, epochs + 1):
    train(model,False,optimizer, epoch)
    test(model,lossv, accv)
end = time.time()
arr = ['Adam - L2']
arr.append(round(float(accv[-1]),2))
arr.append(round(float(end-start),2))
list1.append(arr)

Train Epoch: 1 [0/50000 (0%)]	Loss: 2.305688
Train Epoch: 1 [10000/50000 (20%)]	Loss: 2.297250
Train Epoch: 1 [20000/50000 (40%)]	Loss: 2.307354
Train Epoch: 1 [30000/50000 (60%)]	Loss: 2.301184
Train Epoch: 1 [40000/50000 (80%)]	Loss: 2.301628

Test set: Average loss: 2.3030, Accuracy: 1000.0/10000 (10.00%)

confusion matrix =  [[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [473. 511. 504. 510. 528. 515. 500. 485. 508. 466.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]]
Train Epoch: 2 [0/50000 (0%)]	Loss: 2.301660
Train Epoch: 2 [10000/50000 (20%)]	Loss: 2.302574
Train Epoch: 2 [20000/50000 (40%)]	Loss: 2.

Adagrad, L2 regularisation

In [ ]:
optimizer = torch.optim.Adagrad(model.parameters(), lr=0.01, lr_decay=0.001, weight_decay=0.1, initial_accumulator_value=0)
start = time.time()
lossv, accv = [], []
for epoch in range(1, epochs + 1):
    train(model,False,optimizer, epoch)
    test(model,lossv, accv)
end = time.time()
arr = ['Adagrad - L2']
arr.append(round(float(accv[-1]),2))
arr.append(round(float(end-start),2))
list1.append(arr)

Train Epoch: 1 [0/50000 (0%)]	Loss: 2.301980
Train Epoch: 1 [10000/50000 (20%)]	Loss: 2.297698
Train Epoch: 1 [20000/50000 (40%)]	Loss: 2.300182
Train Epoch: 1 [30000/50000 (60%)]	Loss: 2.300006
Train Epoch: 1 [40000/50000 (80%)]	Loss: 2.302365

Test set: Average loss: 2.3023, Accuracy: 1000.0/10000 (10.00%)

confusion matrix =  [[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [473. 511. 504. 510. 528. 515. 500. 485. 508. 466.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]]
Train Epoch: 2 [0/50000 (0%)]	Loss: 2.302180
Train Epoch: 2 [10000/50000 (20%)]	Loss: 2.302545
Train Epoch: 2 [20000/50000 (40%)]	Loss: 2.

Adadelta, L2 regularistion

In [ ]:
optimizer = torch.optim.Adadelta(model.parameters(), lr=1.0, rho=0.9, eps=1e-06, weight_decay=0.1)
start = time.time()
lossv, accv = [], []
for epoch in range(1, epochs + 1):
    train(model,False,optimizer, epoch)
    test(model,lossv, accv)
end = time.time()
arr = ['Adadelta - L2']
arr.append(round(float(accv[-1]),2))
arr.append(round(float(end-start),2))
list1.append(arr)


Train Epoch: 1 [0/50000 (0%)]	Loss: 2.302641
Train Epoch: 1 [10000/50000 (20%)]	Loss: 2.303349
Train Epoch: 1 [20000/50000 (40%)]	Loss: 2.302990
Train Epoch: 1 [30000/50000 (60%)]	Loss: 2.302214
Train Epoch: 1 [40000/50000 (80%)]	Loss: 2.303760

Test set: Average loss: 2.3027, Accuracy: 1000.0/10000 (10.00%)

confusion matrix =  [[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [473. 511. 504. 510. 528. 515. 500. 485. 508. 466.]]
Train Epoch: 2 [0/50000 (0%)]	Loss: 2.301217
Train Epoch: 2 [10000/50000 (20%)]	Loss: 2.301931
Train Epoch: 2 [20000/50000 (40%)]	Loss: 2.

RMSProp, L2 regularisation

In [ ]:
optimizer = torch.optim.RMSprop(model.parameters(), lr=0.01, alpha=0.99, eps=1e-08, weight_decay=0.1, momentum=0.9, centered=True)
start = time.time()
lossv, accv = [], []
for epoch in range(1, epochs + 1):
    train(model,False,optimizer, epoch)
    test(model,lossv, accv)
end = time.time()
arr = ['RMSProp - L2']
arr.append(round(float(accv[-1]),2))
arr.append(round(float(end-start),2))
list1.append(arr)

Train Epoch: 1 [0/50000 (0%)]	Loss: 2.303222
Train Epoch: 1 [10000/50000 (20%)]	Loss: 2.308527
Train Epoch: 1 [20000/50000 (40%)]	Loss: 2.324510
Train Epoch: 1 [30000/50000 (60%)]	Loss: 2.307116
Train Epoch: 1 [40000/50000 (80%)]	Loss: 2.305226

Test set: Average loss: 2.3035, Accuracy: 1000.0/10000 (10.00%)

confusion matrix =  [[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [473. 511. 504. 510. 528. 515. 500. 485. 508. 466.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]]
Train Epoch: 2 [0/50000 (0%)]	Loss: 2.305955
Train Epoch: 2 [10000/50000 (20%)]	Loss: 2.322601
Train Epoch: 2 [20000/50000 (40%)]	Loss: 2.

Comparison of model accuracy and convergence times wrt different optimizers


In [ ]:
print(tabulate(list1,headers="firstrow"))

Optmizer with Regularisation      Accuracy    Convergence time(sec)
------------------------------  ----------  -----------------------
SGD-Dropout                          46                       63.07
Adam - Dropout                       10                       65.28
Adagrad - Dropout                    20.55                    63.52
Adadelta - Dropout                   11.89                    66.29
RMSProp - Dropout                    10                       65.76
SGD - L1                             45.88                    64.97
Adam - L1                            45.88                    67.04
Adagrad - L1                         45.88                    65.72
Adadelta - L1                        45.88                    68.07
RMSProp - L1                         45.88                    67.72
SGD - L2                              9.72                    62.33
Adam - L2                            10.92                    64.48
Adagrad - L2                         10         

Find out how deep the model can be and still improving

In [ ]:
list2 = [['Depth', 'Accuracy', 'Convergence time(seconds))']]


In [ ]:
class Net3(nn.Module):
    def __init__(self):
        super(Net3, self).__init__()
        self.fc1 = nn.Linear(3*32*32, 2000)
        self.fc2 = nn.Linear(2000, 1000)
        self.fc3 = nn.Linear(1000, 100)
        self.fc4 = nn.Linear(100, 10)
        
        

    def forward(self, x):
        x = x.view(-1, 3*32*32)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        return F.log_softmax(self.fc4(x),dim=1)

model = Net3()
model.to(avDev)
optimizer = torch.optim.SGD(model_l1.parameters(), lr=learning_rate, momentum=momentum, weight_decay=0.1)
start = time.time()
lossv, accv = [], []
for epoch in range(1, epochs + 1):
    train(model,False,optimizer, epoch)
    test(model,lossv, accv)
end = time.time()
arr = ['Depth 3']
arr.append(round(float(accv[-1]),2))
arr.append(round(float(end-start),2))
list2.append(arr)

Train Epoch: 1 [0/50000 (0%)]	Loss: 2.308549
Train Epoch: 1 [10000/50000 (20%)]	Loss: 2.304191
Train Epoch: 1 [20000/50000 (40%)]	Loss: 2.313160
Train Epoch: 1 [30000/50000 (60%)]	Loss: 2.301583
Train Epoch: 1 [40000/50000 (80%)]	Loss: 2.296916

Test set: Average loss: 2.3051, Accuracy: 1034.0/10000 (10.34%)

confusion matrix =  [[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [376. 312. 176. 218. 148. 227. 114. 209. 406. 332.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   2.   0.   0.   0.   1.   0.   1.   0.   1.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [ 97. 197. 328. 292. 380. 287. 386. 275. 102. 133.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]]
Train Epoch: 2 [0/50000 (0%)]	Loss: 2.312432
Train Epoch: 2 [10000/50000 (20%)]	Loss: 2.293064
Train Epoch: 2 [20000/50000 (40%)]	Loss: 2.

In [ ]:
class Net4(nn.Module):
    def __init__(self):
        super(Net4, self).__init__()
        self.fc1 = nn.Linear(3*32*32, 2000)
        self.fc2 = nn.Linear(2000, 1000)
        self.fc3 = nn.Linear(1000, 200)
        self.fc4 = nn.Linear(200, 100)
        self.fc5 = nn.Linear(100, 10)
        
        

    def forward(self, x):
        x = x.view(-1, 3*32*32)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        return F.log_softmax(self.fc5(x),dim=1)

model = Net4()
model.to(avDev)
optimizer = torch.optim.SGD(model_l1.parameters(), lr=learning_rate, momentum=momentum, weight_decay=0.1)
start = time.time()
lossv, accv = [], []
for epoch in range(1, epochs + 1):
    train(model,False,optimizer, epoch)
    test(model,lossv, accv)
end = time.time()
arr = ['Depth 4']
arr.append(round(float(accv[-1]),2))
arr.append(round(float(end-start),2))
list2.append(arr)

Train Epoch: 1 [0/50000 (0%)]	Loss: 2.302472
Train Epoch: 1 [10000/50000 (20%)]	Loss: 2.310904
Train Epoch: 1 [20000/50000 (40%)]	Loss: 2.317552
Train Epoch: 1 [30000/50000 (60%)]	Loss: 2.302845
Train Epoch: 1 [40000/50000 (80%)]	Loss: 2.295877

Test set: Average loss: 2.3058, Accuracy: 1000.0/10000 (10.00%)

confusion matrix =  [[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  3.   0.   1.   0.   0.   6.   1.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [470. 511. 503. 510. 528. 509. 499. 485. 508. 466.]]
Train Epoch: 2 [0/50000 (0%)]	Loss: 2.300143
Train Epoch: 2 [10000/50000 (20%)]	Loss: 2.313546
Train Epoch: 2 [20000/50000 (40%)]	Loss: 2.

In [ ]:
class Net5(nn.Module):
    def __init__(self):
        super(Net5, self).__init__()
        self.fc1 = nn.Linear(3*32*32, 2000)
        self.fc2 = nn.Linear(2000, 1000)
        self.fc3 = nn.Linear(1000, 200)
        self.fc4 = nn.Linear(200, 100)
        self.fc5 = nn.Linear(100, 50)
        self.fc6 = nn.Linear(50, 10)
        
        

    def forward(self, x):
        x = x.view(-1, 3*32*32)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = torch.relu(self.fc5(x))
        return F.log_softmax(self.fc6(x),dim=1)

model = Net5()
model.to(avDev)
optimizer = torch.optim.SGD(model_l1.parameters(), lr=learning_rate, momentum=momentum, weight_decay=0.1)
start = time.time()
lossv, accv = [], []
for epoch in range(1, epochs + 1):
    train(model,False,optimizer, epoch)
    test(model,lossv, accv)
end = time.time()
arr = ['Depth 5']
arr.append(round(float(accv[-1]),2))
arr.append(round(float(end-start),2))
list2.append(arr)

Train Epoch: 1 [0/50000 (0%)]	Loss: 2.300030
Train Epoch: 1 [10000/50000 (20%)]	Loss: 2.307205
Train Epoch: 1 [20000/50000 (40%)]	Loss: 2.302070
Train Epoch: 1 [30000/50000 (60%)]	Loss: 2.322447
Train Epoch: 1 [40000/50000 (80%)]	Loss: 2.324955

Test set: Average loss: 2.3052, Accuracy: 1000.0/10000 (10.00%)

confusion matrix =  [[473. 511. 504. 510. 528. 515. 500. 485. 508. 466.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]]
Train Epoch: 2 [0/50000 (0%)]	Loss: 2.306887
Train Epoch: 2 [10000/50000 (20%)]	Loss: 2.312114
Train Epoch: 2 [20000/50000 (40%)]	Loss: 2.

Table summarising model's depth, accuracy and convergence time

In [ ]:
print(tabulate(list2,headers="firstrow"))

Depth      Accuracy    Convergence time(seconds))
-------  ----------  ----------------------------
Depth 3       10.34                         63.48
Depth 4       10                            64.46
Depth 5       10                            64.27


with tanh, dropout,sgd

In [ ]:
class Net8(nn.Module):
    def __init__(self):
        super(Net8, self).__init__()
        self.fc1 = nn.Linear(3*32*32, 1000)
        self.fc1_drop = nn.Dropout(0.2)
        self.fc2 = nn.Linear(1000, 100)
        self.fc2_drop = nn.Dropout(0.2)
        self.fc3 = nn.Linear(100, 10)

    def forward(self, x):
        x = x.view(-1, 3*32*32)
        x = torch.tanh(self.fc1(x))
        x = self.fc1_drop(x)
        x = torch.tanh(self.fc2(x))
        x = self.fc2_drop(x)
        return F.log_softmax(self.fc3(x),dim=1)

shape = trainset[0][0].numpy().shape
input_dim = shape[0] * shape[1] * shape[2]
output_dim = 10
 
model = Net8()
model.to(avDev)

epochs = 10
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)
arr = ['sgd dropout tanh']
start = time.time()
lossv, accv = [], []
for epoch in range(1, epochs + 1):
    train(model,False,optimizer, epoch)
    test(model,lossv, accv)
end = time.time()
list1 = [['Optmizer with Regularisation','Accuracy','Convergence time(sec)']]
arr.append(round(float(accv[-1]),2))
arr.append(round(float(end-start),2))
list1.append(arr)

Train Epoch: 1 [0/50000 (0%)]	Loss: 2.329339
Train Epoch: 1 [10000/50000 (20%)]	Loss: 2.084622
Train Epoch: 1 [20000/50000 (40%)]	Loss: 1.969517
Train Epoch: 1 [30000/50000 (60%)]	Loss: 2.020555
Train Epoch: 1 [40000/50000 (80%)]	Loss: 1.880899

Test set: Average loss: 1.8865, Accuracy: 3411.0/10000 (34.11%)

confusion matrix =  [[225.  44.  77.  29.  44.  23.   5.  26. 114.  36.]
 [ 15. 138.  16.  21.  13.  14.  13.  18.  29.  54.]
 [ 14.   2.  36.  21.  24.  27.  14.   9.   5.   0.]
 [ 11.   9.  18.  43.  22.  25.  22.  12.   4.  11.]
 [ 15.  15. 139.  56. 189.  60.  70.  90.   4.   6.]
 [ 22.  38.  61. 151.  59. 211.  83.  49.  46.  20.]
 [ 10.  45.  73.  63.  96.  72. 207.  42.   1.  30.]
 [ 39.  25.  50.  76.  54.  57.  53. 180.  15.  28.]
 [ 97.  79.  22.  15.  16.  16.   8.  22. 235.  67.]
 [ 25. 116.  12.  35.  11.  10.  25.  37.  55. 214.]]
Train Epoch: 2 [0/50000 (0%)]	Loss: 2.001437
Train Epoch: 2 [10000/50000 (20%)]	Loss: 1.908021
Train Epoch: 2 [20000/50000 (40%)]	Loss: 1.

with sigmoid, dropout, sgd


In [ ]:
class Net9(nn.Module):
    def __init__(self):
        super(Net9, self).__init__()
        self.fc1 = nn.Linear(3*32*32, 1000)
        self.fc1_drop = nn.Dropout(0.2)
        self.fc2 = nn.Linear(1000, 100)
        self.fc2_drop = nn.Dropout(0.2)
        self.fc3 = nn.Linear(100, 10)

    def forward(self, x):
        x = x.view(-1, 3*32*32)
        x = torch.sigmoid(self.fc1(x))
        x = self.fc1_drop(x)
        x = torch.sigmoid(self.fc2(x))
        x = self.fc2_drop(x)
        return F.log_softmax(self.fc3(x),dim=1)

shape = trainset[0][0].numpy().shape
input_dim = shape[0] * shape[1] * shape[2]
output_dim = 10
 
model = Net9()
model.to(avDev)

epochs = 10
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)
arr = ['sgd dropout sigmoid']
start = time.time()
lossv, accv = [], []
for epoch in range(1, epochs + 1):
    train(model,False,optimizer, epoch)
    test(model,lossv, accv)
end = time.time()

arr.append(round(float(accv[-1]),2))
arr.append(round(float(end-start),2))
list1.append(arr)


Train Epoch: 1 [0/50000 (0%)]	Loss: 2.360305
Train Epoch: 1 [10000/50000 (20%)]	Loss: 2.326370
Train Epoch: 1 [20000/50000 (40%)]	Loss: 2.311776
Train Epoch: 1 [30000/50000 (60%)]	Loss: 2.313972
Train Epoch: 1 [40000/50000 (80%)]	Loss: 2.295538

Test set: Average loss: 2.2953, Accuracy: 1098.0/10000 (10.98%)

confusion matrix =  [[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [ 23.  12.   1.   1.   0.   0.   0.   2.  50.  26.]
 [ 42.  30.  18.  15.   3.   9.   9.  12.  19.  16.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [407. 451. 484. 492. 525. 506. 490. 467. 435. 395.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   1.   0.   0.   0.   0.   0.   1.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  1.  18.   1.   1.   0.   0.   1.   4.   4.  28.]]
Train Epoch: 2 [0/50000 (0%)]	Loss: 2.284042
Train Epoch: 2 [10000/50000 (20%)]	Loss: 2.294226
Train Epoch: 2 [20000/50000 (40%)]	Loss: 2.

Comparison of model accuracy and convergence times for tanh and sigmoid activation functions

In [ ]:
print(tabulate(list1,headers="firstrow"))

Optmizer with Regularisation      Accuracy    Convergence time(sec)
------------------------------  ----------  -----------------------
sgd dropout tanh                     42.45                    63.29
sgd dropout sigmoid                  27.24                    63.24
